---
title: Basic App
format: html
filters:
  - shinylive
jupyter: python3
draft: true
---


```{shinylive-python}
#| standalone: true
#| components: [editor, viewer]
## file: app.py
from shiny import App, render, ui
import sys
sys.path.append('.')
import mini_mesa as mm

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B

class MoneyModel(Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mm.space.MultiGrid(width, height, True)
        self.schedule = mm.time.RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mm.DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": "wealth", "Steps_not_given": "steps_not_given"},
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


class MoneyAgent(Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1
        self.steps_not_given = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            self.steps_not_given = 0
        else:
            self.steps_not_given += 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()
        else:
            self.steps_not_given += 1

app_ui = ui.page_fluid(
    ui.input_slider("agents", "Agents", 0, 100, 100),
    ui.input_slider("steps", "Steps", 0, 100, 100),
    ui.input_slider("x", "y", 0, 10, 10),
    ui.input_slider("y", "y", 0, 10, 10),
    ui.output_text_verbatim("txt"),
)

def server(input, output, session):
    @output
    @render.text    
    def txt():
      params = {"width": 10, "height": 10, "N": range(5, 100, 5)}

      params['N']=input.agents(),
      params['N']=input.steps(),
      

      results = mm.batch_run(
        MoneyModel,
        parameters=params,
        iterations=7,
        max_steps=100,
        number_processes=1,
        data_collection_period=1,
        display_progress=True,
      )

      
      results = model.run()
      return results.info['completed_steps'] 

## file: requirements.txt
pandas
```
:::